In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import json
import time
driver = webdriver.Chrome('./chromedriver.exe')

In [2]:
name = '쯔양' #input()

In [3]:
url = 'https://some.co.kr/analysis/social/reputation'
driver.get(url)

In [4]:
# 팝업창 닫기
WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, "/html/body/div[13]/div[5]/div/p/a"))).click()

In [5]:
# 모달 열기
driver.find_element_by_xpath("//button[@type='button']").click()

In [6]:
# 검색 시작
find = driver.find_element_by_xpath("/html/body/div[4]/div/div[1]/div[2]/div[1]/form/fieldset/div/div[1]/div/input")
find.send_keys(name)
driver.find_element_by_xpath("/html/body/div[4]/div/div[1]/div[2]/div[2]/div/div[2]/button").click()
time.sleep(2)

In [7]:
from bs4 import BeautifulSoup
from html_table_parser import parser_functions as parser

In [8]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [9]:
# 댓글 평판 크롤링
temp = soup.find_all('table')
p = parser.make2d(temp[1])

In [10]:
positive = [ [x[0],x[2]] for x in p if x[1]=='긍정']
negative = [ [x[0],x[2]] for x in p if x[1]=='부정']
neutral = [ [x[0],x[2]] for x in p if x[1]=='중립']

In [11]:
data = {}
data['name'] = name

In [12]:
data['reputation'] = []
data["reputation"].append({
    "type" : "positive",
    "data" : positive
})
data["reputation"].append({
    "type" : "negative",
    "data" : negative
})
data["reputation"].append({
    "type" : "neutral",
    "data" : neutral
})

In [13]:
data

{'name': '쯔양',
 'reputation': [{'type': 'positive',
   'data': [['이쁘다', '29'],
    ['응원하다', '29'],
    ['먹고싶다', '27'],
    ['맛있다', '26'],
    ['좋다', '21'],
    ['유명하다', '19'],
    ['좋아하다', '18'],
    ['즐기다', '15'],
    ['명성 자자하다', '12'],
    ['감칠맛', '11'],
    ['손맛', '11'],
    ['웃음', '11'],
    ['진심', '10'],
    ['잘먹다', '10'],
    ['흥행', '10'],
    ['감사드리다', '8'],
    ['부럽다', '7'],
    ['신기하다', '6'],
    ['웃다', '5'],
    ['신선하다', '5'],
    ['손님 많다', '4'],
    ['가고싶다', '4'],
    ['대단하다', '4'],
    ['재미있다', '4']]},
  {'type': 'negative',
   'data': [['죄송하다', '31'], ['정 떨어지다', '24'], ['덥다', '6'], ['정떨어지다', '5']]},
  {'type': 'neutral', 'data': [['달다', '14'], ['배부르다', '4']]}]}

In [14]:
with open("./sometred.json", 'w',encoding="UTF-8") as outfile:
    json.dump(data, outfile,ensure_ascii = False)